In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

# 台灣銀行特定年月份匯率
def read_currency(year,month,currency):
    date  = datetime.date(year,month,1)
    date  = datetime.datetime.strftime(date,"%Y-%m")
    url = 'https://rate.bot.com.tw/xrt/quote/'+date+'/'+currency+'?Lang=en-US'
    time.sleep(7)
    df = pd.read_html(url)[0]
    df = df.iloc[:,[0,2,3,4,5]]
    df.columns=['Date','Cash Rate Buying','Cash Rate Selling','Spot Rate Buying','Spot Rate Selling']
    return df

# 特定年月份大盤資訊
def read_market(year,month):
    date  = datetime.date(year,month,1)
    date  = datetime.datetime.strftime(date,"%Y%m%d")
    url = 'https://www.twse.com.tw/en/exchangeReport/FMTQIK?response=html&date='+date
    time.sleep(7)
    try:
        df = pd.read_html(url)[0]     
        df.columns = df.columns.get_level_values(1)
    except:
        print('No Data at '+str(year)+'/'+str(month))
        df = None
    return df

# 特定股票特定年月份股價
def read_single_stock_price(year,month,stockNo):
    date  = datetime.date(year,month,1)
    date  = datetime.datetime.strftime(date,"%Y%m%d")
    url = 'https://www.twse.com.tw/en/exchangeReport/STOCK_DAY?response=html&date='+date+'&stockNo='+stockNo
    time.sleep(7)
    try:
        df = pd.read_html(url)[0]     
        df.columns = df.columns.get_level_values(1)
    except:
        print('No Data at '+str(year)+'/'+str(month))
        df = None
    return df

# 特定股票所有股價
def read_history_single_stock_price(year,month,day,stockNo):
    date    = str(int(time.mktime(datetime.datetime(year,month,day).timetuple())))
    url = 'https://query1.finance.yahoo.com/v7/finance/download/'+stockNo+'.TW?period1=0&period2='+date+'&interval=1d&events=history&crumb=hP2rOschxO0'
    time.sleep(1)
    try:
        df = pd.read_csv(url)
        df = df.drop(columns=['Adj Close'])
        df.columns = ['Date','Opening Price','Highest Price','Lowest Price','Closing Price','Trade Volume']
        df = df[df['Closing Price'].isnull() != True]
    except:
        print('Error')
        df = None
    return df
    
# 特定股票特定年月份 P/B P/E
def read_single_stock_ratio(year,month,stockNo):
    date  = datetime.date(year,month,1)
    date  = datetime.datetime.strftime(date,"%Y%m%d")
    url = 'https://www.twse.com.tw/en/exchangeReport/BWIBBU?response=html&date='+date+'&stockNo='+stockNo
    time.sleep(7)
    try:
        df = pd.read_html(url)[0]     
        df.columns = df.columns.get_level_values(1)
    except:
        print('No Data at '+str(year)+'/'+str(month))
        df = None
    return df

# 特定日所有指數與股價
def read_all_stock(year,month,day):
    date  = datetime.date(year,month,day)
    date  = datetime.datetime.strftime(date,"%Y%m%d")
    url = 'http://www.twse.com.tw/en/exchangeReport/MI_INDEX?response=html&date='+date+'&type=ALL'
    try:
        Index = pd.read_html(url)[0]
        Index.columns = Index.columns.get_level_values(1)
        Index = Index.dropna(axis='columns')
        Return_Index  = pd.read_html(url)[3]
        Return_Index.columns = Return_Index.columns.get_level_values(1)
        Return_Index  = Return_Index.dropna(axis='columns')
        Stock_Price   = pd.read_html(url)[8]
        Stock_Price.columns = Stock_Price.columns.get_level_values(2)
    except:
        print('No Data at '+str(year)+'/'+str(month))
        Index,Return_Index,Stock_Price = None,None,None
    return Index,Return_Index,Stock_Price
    


In [ ]:
# Read Stock Price from Yahoo Finance
target = ['2881','2884','2890','5880','2882','2812','2618','2633','2330','2888','2610']
target = ['0050','0056']

for stockNo in target:
    print("Read "+stockNo+" Price from Yahoo Finance")
    history_data = read_history_single_stock_price(2020,3,18,stockNo)
    history_data.to_csv('./Data/'+stockNo +'_Price.csv',index=0)

print("Finished")

In [ ]:
# Read Stock Price from TWSE
target = ['2881','2884','2890','5880','2882','2812','2618','2633','2330','2888','2610']

for stockNo in target:
    print("Read "+stockNo+" Price from TWSE")
    df = read_single_stock_price(2016,12,stockNo)
    
    for year in range(3):
        for month in range(12): 
            df1 = read_single_stock_price(2017+year,month+1,stockNo)
            df = df.append(df1, ignore_index=True)

    df1 = read_single_stock_price(2020,1,stockNo)
    df = df.append(df1, ignore_index=True)
    df1 = read_single_stock_price(2020,2,stockNo)
    df = df.append(df1, ignore_index=True)
    df1 = read_single_stock_price(2020,3,stockNo)
    df = df.append(df1, ignore_index=True)

    history_data = df[df['Closing Price'] != '--']
    history_data.to_csv('./Data/'+stockNo +'_Price.csv',index=0)

print("Finished")

In [ ]:
# Read Stock Ratio from TWSE
for stockNo in target:
    print("Read "+stockNo+" Ratio from TWSE")
    df = read_single_stock_ratio(2016,12,stockNo)

    for year in range(3):
        for month in range(12): 
            df1 = read_single_stock_ratio(2017+year,month+1,stockNo)
            df = df.append(df1, ignore_index=True)

    df1 = read_single_stock_ratio(2020,1,stockNo)
    df = df.append(df1, ignore_index=True)
    df1 = read_single_stock_ratio(2020,2,stockNo)
    df = df.append(df1, ignore_index=True)

    history_data = df[df['P/E ratio'] != '-']
    history_data.to_csv('./Data/'+stockNo +'_Ratio.csv',index=0)

print("Finished")

In [ ]:
# Read Single Currency 
currency = 'USD'
df = read_currency(2019,1,currency)

for month in range(11): 
    df1 = read_currency(2019,month+2,currency)
    df = df.append(df1, ignore_index=True)
        
df1 = read_currency(2020,1,currency)
df = df.append(df1, ignore_index=True)
df1 = read_currency(2020,2,currency)
df = df.append(df1, ignore_index=True)

history_data = df[df['Spot Rate Selling'] != '--']
history_data.to_csv('./Data/'+currency +'_Currency.csv',index=0)

In [ ]:
# Read Index from TWSE
df = read_market(2016,12)

for year in range(3):
    for month in range(12): 
        df1 = read_market(2017+year,month+1)
        df = df.append(df1, ignore_index=True)
        
df1 = read_market(2020,1)
df = df.append(df1, ignore_index=True)
df1 = read_market(2020,2)
df = df.append(df1, ignore_index=True)

history_data = df[df['Transaction'] != '--']
history_data.to_csv('./Data/Market_Info.csv',index=0)

In [ ]:
Index,Return_Index,Stock_Price = read_all_stock(date)

In [ ]:
stockNo = '2888'
currency = 'USD' 
stock_price = pd.read_csv('./Data/'+stockNo+'_Price.csv') 
stock_ratio = pd.read_csv('./Data/'+stockNo+'_Ratio.csv') 
market_info = pd.read_csv('./Data/Market_Info.csv')    
currency_info = pd.read_csv('./Data/'+currency +'_Currency.csv')  
stock_price.plot(x='Date', y=['Highest Price','Lowest Price','Opening Price','Closing Price'],title=stockNo + ' History Price',grid=True, subplots=False,figsize=(16,6))
stock_ratio.plot(x='Date', y=['P/B ratio','P/E ratio'],kind='line',title=stockNo + ' History Ratio',grid=True,subplots=True,figsize=(16,6))
market_info.plot(x='Date', y=['TAIEX','Change'],kind='line',title='Market Information',grid=True,subplots=True,figsize=(16,6))
currency_info.plot(x='Date', y=['Spot Rate Buying','Spot Rate Selling'],kind='line',title='Currency Information',grid=True,subplots=False,figsize=(16,4))

In [ ]:
Index,Return_Index,Stock_Price = read_all_stock(2020,3,23)